# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases   Deaths    PCases  PDeaths  Cases (+)  \
0               US  97977357  1074485  97741415  1072664     235942   
1            India  44664810   530520  44660579   530500       4231   
2           France  37288432   158711  37140238   158317     148194   
3          Germany  36005025   155403  35823771   154535     181254   
4           Brazil  34889576   688607  34849063   688332      40513   
5      South Korea  26091539    29571  25838239    29372     253300   
6   United Kingdom  24179611   211415  24155154   210680      24457   
7            Italy  23642011   179436  23642011   179436          0   
8            Japan  22981382    47442  22635975    47087     345407   
9           Russia  21178202   382949  21154149   382644      24053   
10          Turkey  16919638   101203  16919638   101203          0   
11           Spain  13529643   115239  13529643   115239          0   
12         Vietnam  11507540    43166  11505608    43165       1932   
13       Australia  10458954    15855  10418986    15786      39968   
14       Argentina   9720232   130003   9718875   129991       1357   

    Deaths (+)  Fatality Rate      Continent  
0         1821            1.1  North America  
1           20            1.2           Asia  
2          394            0.4         Europe  
3          868            0.4         Europe  
4          275            2.0  South America  
5          199            0.1           Asia  
6          735            0.9         Europe  
7            0            0.8         Europe  
8          355            0.2           Asia  
9          305            1.8         Europe  
10           0            0.6           Asia  
11           0            0.9         Europe  
12           1            0.4           Asia  
13          69            0.2        Oceania  
14          12            1.3  South America

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2022-11-10 00:00:00'),
 'since': Timestamp('2022-11-05 00:00:00'),
 'Cases': 633987450,
 'Deaths': 6604549,
 'Cases (+)': 1822361,
 'Deaths (+)': 7548,
 'China Cases': 3053673,
 'China Deaths': 15733,
 'China Cases (+)': 56367,
 'China Deaths (+)': 44,
 'US Cases': 97977357,
 'US Deaths': 1074485,
 'US Cases (+)': 235942,
 'US Deaths (+)': 1821,
 'EU Cases': 236774625,
 'EU Deaths': 1964173,
 'EU Cases (+)': 553028,
 'EU Deaths (+)': 3463}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  11/1/22  11/2/22  11/3/22  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...      104       98      130   
Albania               0        0        0  ...        3       27        0   
Algeria               0        0        0  ...        1        1        7   
Andorra               0        0        0  ...        0       53        0   
Angola                0        0        0  ...        0        0        0   

                11/4/22  11/5/22  11/6/22  11/7/22  11/8/22  11/9/22  11/10/22  
Country/Region                                                                  
Afghanistan         102       77      107      148      113      152       193  
Albania              31       19        9        3       13       17        15  
Algeria               9        6       11        8       10       15        11  
Andorra               0        0        0        0        0       76         0  
Angola                0        0        0        0        0        0         0  

[5 rows x 1024 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)